In [2]:
import pandas as pd
import numpy as np

In [3]:
# Reading Order excel sheet

order = pd.read_excel(r"Company ShopX - Order Report.xlsx")

In [4]:
# Reading Product excel sheet

product = pd.read_excel(r"Company ShopX - Product Weight.xlsx")

In [8]:
# merging order and product table

rder_product = pd.merge(left = order, right= product, how = "inner", on = "Product Code")
order_product

,Order ID,Product Code,Units Ordered,Product Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127
...,...,...,...,...
396,2001806616,8904223819123,1.0,250
397,2001806567,8904223815804,1.0,160
398,2001806567,8904223818577,1.0,150
399,2001806408,8904223819437,2.0,552


In [10]:
# assigning total weight column 

order_product["Total weight as per ShopX (KG)"] = round((order_product["Units Ordered"] * order_product["Product Weight (g)"])/1000,2)

In [12]:
# Selecting important columns from order + product table

rev_order_product = order_product.iloc[:,[0,4]]

In [14]:
# Grouping repeted Order ID

order_group = rev_order_product.groupby("Order ID")[["Total weight as per ShopX (KG)"]].sum()

In [16]:
# Weight slab function

def slab(x):
    return np.ceil(x*2)/2

In [18]:
# assigning weight slab column according to ShopX

order_group["Weight slab as per ShopX (KG)"] = order_group["Total weight as per ShopX (KG)"].apply(slab)

In [20]:
# read invoice excel file and selecting important columns from invoice table 

extra_info = pd.read_excel(r"Courier Company - Invoice.xlsx").iloc[:,[0,1,2,3,4,5,6]]
extra_info

,AWB Code (Airway Bill Number),Order ID,Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone,Freight Type
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges
...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges


In [22]:
# Merging (order + product) table with courier company invoice table

before_amount = pd.merge(left = order_group, right= extra_info, how = "inner", on = "Order ID")
before_amount

,Order ID,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone,Freight Type
0,2001806210,0.22,0.5,1091117221940,2.92,121003,140604,b,Forward charges
1,2001806226,0.48,0.5,1091117222065,0.68,121003,723146,d,Forward charges
2,2001806229,0.50,0.5,1091117222080,0.71,121003,421204,d,Forward charges
3,2001806232,1.30,1.5,1091117222124,1.30,121003,507101,d,Forward charges
4,2001806233,0.25,0.5,1091117222135,0.78,121003,263139,b,Forward charges
...,...,...,...,...,...,...,...,...,...
119,2001821995,0.48,0.5,1091121183730,0.50,121003,342008,d,Forward charges
120,2001822466,1.38,1.5,1091121305541,1.10,121003,342301,d,Forward charges
121,2001823564,0.66,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges
122,2001825261,1.56,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges


In [24]:
# creating rate function according to rate charged by Courier company

amount = []

for i in range(before_amount.shape[0]):
    
    total_amount = 0
    
    if (before_amount["Delivery Zone"][i] == "a") and (before_amount["Freight Type"][i] == "Forward charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 29.5
        additional = (total_slab - 1) * 23.6
        total_amount = fixed + additional
        amount.append(total_amount)
        
    elif (before_amount["Delivery Zone"][i] == "b") and (before_amount["Freight Type"][i] == "Forward charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 33
        additional = (total_slab - 1) * 28.3
        total_amount = fixed + additional
        amount.append(total_amount)

    elif (before_amount["Delivery Zone"][i] == "c") and (before_amount["Freight Type"][i] == "Forward charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 40.1
        additional = (total_slab - 1) * 38.9
        total_amount = fixed + additional
        amount.append(total_amount)
        
    elif (before_amount["Delivery Zone"][i] == "d") and (before_amount["Freight Type"][i] == "Forward charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 45.4
        additional = (total_slab - 1) * 44.8
        total_amount = fixed + additional
        amount.append(total_amount)

    elif (before_amount["Delivery Zone"][i] == "e") and (before_amount["Freight Type"][i] == "Forward charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 56.6
        additional = (total_slab - 1) * 55.5
        total_amount = fixed + additional
        amount.append(total_amount)
        
    elif (before_amount["Delivery Zone"][i] == "a") and (before_amount["Freight Type"][i] == "Forward and RTO charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 13.6
        additional = (total_slab - 1) * 23.6
        total_amount = fixed + additional
        amount.append(total_amount)
        
    elif (before_amount["Delivery Zone"][i] == "b") and (before_amount["Freight Type"][i] == "Forward and RTO charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 20.5
        additional = (total_slab - 1) * 28.3
        total_amount = fixed + additional
        amount.append(total_amount)
    
    elif (before_amount["Delivery Zone"][i] == "c") and (before_amount["Freight Type"][i] == "Forward and RTO charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 31.9
        additional = (total_slab - 1) * 38.9
        total_amount = fixed + additional
        amount.append(total_amount)
    
    elif (before_amount["Delivery Zone"][i] == "d") and (before_amount["Freight Type"][i] == "Forward and RTO charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 41.3
        additional = (total_slab - 1) * 44.8
        total_amount = fixed + additional
        amount.append(total_amount)
        
    elif (before_amount["Delivery Zone"][i] == "e") and (before_amount["Freight Type"][i] == "Forward and RTO charges"):
        total_slab = int(before_amount["Weight slab as per ShopX (KG)"][i] / 0.5)
        fixed = 1 * 50.7
        additional = (total_slab - 1) * 55.5
        total_amount = fixed + additional
        amount.append(total_amount)

In [26]:
# Assigning Amount Column according ShopX

before_amount["Expected Charge as per ShopX (Rs.)"] = amount

In [28]:
# Assigning Weight Slab according to ShopX

before_amount["Weight slab charged by Courier Company (KG)"] = before_amount["Chargeable Weight"].apply(slab)

In [30]:
# Information analysis by ShopX

after_amount = before_amount.iloc[:,[0,3,1,2,4,10,7,9]].rename(columns ={"Chargeable Weight": "Total weight as per Courier Company (KG)", "Delivery Zone": "Delivery Zone as per ShopX" })

In [32]:
# Reading Invoice Table

invoice = pd.read_excel(r"Courier Company - Invoice.xlsx")

In [34]:
# joining invoice table and information table according to ShopX

final_output = pd.merge(left= after_amount, right= invoice, how ="inner", on= "Order ID").iloc[:,[0,1,2,3,4,5,6,12,7,14]].rename(columns={"Delivery Zone": "Delivery Zone charged by Courier Company", "Total Amount (Rs.)" : "Charges Billed by Courier Company (Rs.) "})

In [65]:
# assigning difference in the amount charged by courier and expected amount of shopX

final_output["Difference Between Expected Charges and Billed Charges (Rs.)"] = round((final_output["Expected Charge as per ShopX (Rs.)"] - final_output["Charges Billed by Courier Company (Rs.) "]),2)

,Order ID,AWB Code (Airway Bill Number) _x,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per ShopX,Delivery Zone charged by Courier Company,Expected Charge as per ShopX (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.48,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.50,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.30,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.25,0.5,0.78,1.0,b,b,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.48,0.5,0.50,0.5,d,d,45.4,45.4,0.0
120,2001822466,1091121305541,1.38,1.5,1.10,1.5,d,d,135.0,135.0,0.0
121,2001823564,1091121666133,0.66,1.0,0.70,1.0,d,d,86.1,172.8,-86.7
122,2001825261,1091121981575,1.56,2.0,1.60,2.0,d,d,175.7,345.0,-169.3


In [104]:
# Separate under charged data

under_charged = final_output[final_output["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0]

In [98]:
# Separate over charged data

over_charged = final_output[final_output["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0] 

In [75]:
# Separate correctly charged data

correctly_charged = final_output[final_output["Difference Between Expected Charges and Billed Charges (Rs.)"] == 0] 

In [108]:
# Summary data of our analysis

summary_data = ((correctly_charged["Order ID"].count(), round(correctly_charged["Expected Charge as per ShopX (Rs.)"].sum(),2)),
(over_charged["Order ID"].count(), round(over_charged["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(),2)),
(under_charged["Order ID"].count(), round(under_charged["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(),2))
               )

((48, 4203.3), (68, -4472.0), (8, 292.4))

In [116]:
# creating dataframe for summary data

summary = pd.DataFrame(summary_data,index=["Total Orders - Correctly Charged","Total Orders - Over Charged","Total Orders - Under Charged"],
            columns= ["Count","Amount"])

In [126]:
# coverting dataframe into excel

blank = pd.ExcelWriter("Results.xlsx")
summary.to_excel(blank, sheet_name="Summary")
final_output.to_excel(blank, sheet_name="Calculation", index=False )
blank.close()